<a href="https://colab.research.google.com/github/alerosenberg/rais_ale/blob/main/IPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget 
!pip install py7zr
!pip install dask
!pip install dask[dataframe]
#!pip install sys
!pip install tqdm
#!pip install os
!pip install pandas
!pip install numpy
!pip install scipy
!pip install matplotlib
!pip install seaborn 
!pip install deflatebr

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=b583c5045a84549c3542889db34074b21cd5070d4a136cb96079faa1c7850f04
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 13.7MB 241kB/s 
     |████████████████████████████████| 92kB 2.4MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=5b2a3071ec34a7eaeaf0fbda32e2f3c809b71c07f220aa43b6f00e9f8b5d92de
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built locket
     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 245kB 8.3MB/s 


In [2]:
import wget
import py7zr
import dask 
import dask.dataframe as dd
import sys
from tqdm import tqdm
import py7zr
import os
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from statsmodels.graphics.factorplots import interaction_plot
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import matplotlib.pyplot as plt
import seaborn as sns
from deflatebr import deflate

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#STEP-1: Import libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
#STEP-2: Autheticate E-Mail ID
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [99]:
from google.colab import files
uploaded = files.upload()

Saving COLAB_IPCA.xlsx to COLAB_IPCA (1).xlsx


In [100]:
import io
ipca = pd.read_excel(io.BytesIO(uploaded['COLAB_IPCA.xlsx']))
ipca

,PRODUTO,ANO,RJ,SP,NACIONAL
0,ABACATE,200204,0.00,0.00,0.96
1,ABACATE,200205,0.00,0.00,0.51
2,ABACATE,200206,0.00,0.00,-0.43
3,ABACATE,200207,0.00,0.00,-0.36
4,ABACATE,200208,0.00,0.00,0.38
...,...,...,...,...,...
98539,VITAMINA E FORTIFICANTE,201208,1.14,-0.17,0.22
98540,VITAMINA E FORTIFICANTE,201209,-0.18,-1.40,-0.33
98541,VITAMINA E FORTIFICANTE,201210,0.42,0.59,-0.10
98542,VITAMINA E FORTIFICANTE,201211,-0.11,1.38,0.29


In [105]:
ipcas = ipca.sort_values(by=['PRODUTO','ANO'], ascending=[True, True])
ipcas.iloc[130:140]



,PRODUTO,ANO,RJ,SP,NACIONAL
130,ABACATE,201208,0.00,0.00,-5.76
131,ABACATE,201209,0.00,0.00,0.31
132,ABACATE,201210,0.00,0.00,2.32
133,ABACATE,201211,0.00,0.00,6.51
134,ABACATE,201212,0.00,0.00,22.94
135,ABACAXI,200204,7.03,-3.06,2.37
136,ABACAXI,200205,-0.88,-6.18,-4.50
137,ABACAXI,200206,-1.23,-4.77,-3.04
138,ABACAXI,200207,9.35,2.12,-0.14
139,ABACAXI,200208,-10.24,-2.76,-3.97


In [107]:
somar_12m = ipcas[['SP']].groupby(ipcas['PRODUTO']).apply(lambda g: g.rolling(12).sum())
# somar_12m.iloc[130:155]


SP
PRODUTO            
ABACATE  130    NaN
         131    NaN
         132    NaN
         133    NaN
         134    NaN
ABACAXI  135    NaN
         136    NaN
         137    NaN
         138    NaN
         139    NaN
         140    NaN
         141    NaN
         142    NaN
         143    NaN
         144    NaN
         145    NaN
         146  16.48
         147  18.58
         148  24.77
         149  19.74
         150  10.80
         151  19.37
         152  17.34
         153  26.77
         154  13.41

In [ ]:
quebras = list(ipca['PRODUTO'].unique())
for quebras in quebras:
  ipca[ipca['PRODUTO'] == quebras]['SP'].rolling(12).sum()
list(ipca['SP'])

In [7]:
!pip install db-sqlite3
import sqlite3
conn = sqlite3.connect('example.db')

     |████████████████████████████████| 174kB 2.6MB/s 
  Created wheel for db-sqlite3: filename=db_sqlite3-0.0.1-cp36-none-any.whl size=1797 sha256=905f36450a79b9690ca6d4f127f3b18da31c5812216f6711faba5e27a61fa6cc
  Stored in directory: /root/.cache/pip/wheels/e0/f8/cb/08e3a3a63bdea9ac3a0cccd140636b6cf94e3fb7df1070b12e
  Created wheel for db: filename=db-0.1.1-cp36-none-any.whl size=3894 sha256=3e29c174bc523ab14f4ac1d0daefa3a7688d9a6c0f08673f424714d64d363897
  Stored in directory: /root/.cache/pip/wheels/30/eb/ba/237fa002d1d1b2e73cedcefd26a9db37c4b72c7e5156ea0501
  Created wheel for antiorm: filename=antiorm-1.2.1-cp36-none-any.whl size=31664 sha256=13f9a8d025eb303a861ce18d6d5f77165a060699a6efde6d2cf4cd99bdb08842
  Stored in directory: /root/.cache/pip/wheels/b0/91/4d/f8fe808786ff1cda9e7e99e1b1bbda9196ab26786017965313
Successfully built db-sqlite3 db antiorm


In [8]:
ipca.to_sql('ipca_base',conn,if_exists='replace',index=False)

In [9]:
sql = ''' select 
a.'PRODUTO' as PRODUTO
,a.'ANO' as ANO
,count(*) as cont
,sum(a.'RJ') as RJ
,sum(a.'SP') as SP
from ipca_base as a
where PRODUTO = 'PRODUTOS FARMACÊUTICOS'
group by 1,2
ORDER BY 'PRODUTO', 'ANO'

'''
pd.read_sql(sql,conn)

,PRODUTO,ANO,cont,RJ,SP
0,PRODUTOS FARMACÊUTICOS,200204,1,1.03,0.48
1,PRODUTOS FARMACÊUTICOS,200205,1,0.24,-0.11
2,PRODUTOS FARMACÊUTICOS,200206,1,-0.36,-0.41
3,PRODUTOS FARMACÊUTICOS,200207,1,-0.16,0.26
4,PRODUTOS FARMACÊUTICOS,200208,1,0.50,0.30
...,...,...,...,...,...
113,PRODUTOS FARMACÊUTICOS,201109,1,-0.53,-0.48
114,PRODUTOS FARMACÊUTICOS,201110,1,0.10,-0.50
115,PRODUTOS FARMACÊUTICOS,201111,1,0.06,0.50
116,PRODUTOS FARMACÊUTICOS,201112,1,-0.06,0.07
